In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE164883'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic heterogeneity of Coeliac Disease biopsies from the duodenum"
!Series_summary	"Here, we present a high-resolution analysis of the transcriptomes extracted from duodenal probes of 25 children and adolescents with active CD and 21 children without CD but with diverse intestinal afflictions as controls. We found that the transcriptomes of CD patients divide into three subgroups, a mixed group resembling part of control cases and a CD-low and CD-high groups referring to lower and higher levels of CD severity"
!Series_summary	"Despite generally increased inflammation, considerable variation in inflammation-level between subgroups was observed, which was further de-chiffred into immune cell types using immune cell de-convolution"
!Series_overall_design	"one biopsy (15–20 mg) was taken from descending duodenum of each patient. Fresh tissue samples were snap frozen and stored in liquid nitrogen until preparation. Frozen biopsies were disrupt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# 2.1 Data Availability
trait_row = 0 if len(set(['Control', 'Celiac disease'])) > 1 else None
age_row = 2 
gender_row = None  # gender data is not provided in this dataset

# 2.3 Data Type Conversion
def extract_value(cell):
    return cell.split(':')[-1].strip()

def convert_trait(cell):
    value = extract_value(cell)
    return 1 if value == 'Celiac disease' else 0 if value == 'Control' else None

def convert_age(cell):
    value = extract_value(cell)
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(cell):
    return None  # Not applicable in this dataset

# Step 3: Save Metadata
save_cohort_info('GSE164883', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Celiac_Disease/trait_data/GSE164883.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5022362': [0.0, 1.0], 'GSM5022363': [1.0, 3.0], 'GSM5022364': [1.0, 3.0], 'GSM5022365': [1.0, 4.0], 'GSM5022366': [1.0, 7.0], 'GSM5022367': [0.0, 6.0], 'GSM5022368': [1.0, 9.0], 'GSM5022369': [0.0, 10.0], 'GSM5022370': [1.0, 10.0], 'GSM5022371': [0.0, 11.0], 'GSM5022372': [1.0, 11.0], 'GSM5022373': [1.0, 12.0], 'GSM5022374': [0.0, 12.0], 'GSM5022375': [0.0, 12.0], 'GSM5022376': [0.0, 12.0], 'GSM5022377': [0.0, 13.0], 'GSM5022378': [1.0, 13.0], 'GSM5022379': [1.0, 13.0], 'GSM5022380': [0.0, 14.0], 'GSM5022381': [0.0, 15.0], 'GSM5022382': [1.0, 16.0], 'GSM5022383': [1.0, 16.0], 'GSM5022384': [0.0, 17.0], 'GSM5022385': [1.0, 17.0], 'GSM5022386': [0.0, 17.0], 'GSM5022387': [1.0, 3.0], 'GSM5022388': [0.0, 2.0], 'GSM5022389': [0.0, 2.0], 'GSM5022390': [1.0, 3.0], 'GSM5022391': [1.0, 4.0], 'GSM5022392': [1.0, 4.0], 'GSM5022393': [1.0, 5.0], 'GSM5022394': [1.0, 4.0], 'GSM5022395': [0.0, 7.0], 'GSM5022396': [1.0, 8.0], 'GSM5022397': [1.0, 12.0], 'GSM5022398': [0.0, 12.0], 'GSM5022399': [1.

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for probe IDs and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and obtain the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Celiac_Disease/gene_data/GSE164883.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Celiac_Disease')

# 4. Save the cohort information.
save_cohort_info('GSE164883', './preprocessed/Celiac_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Celiac_Disease/GSE164883.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Celiac_Disease', the least common label is '0.0' with 22 occurrences. This represents 45.83% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 4.75
  50% (Median): 12.0
  75%: 14.0
Min: 1.0
Max: 17.0
The distribution of the feature 'Age' in this dataset is fine.

